In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
from aurora import Batch, Metadata, AuroraSmallPretrained
from datetime import datetime, timedelta
from torch.nn import L1Loss
import torch.optim as optim
import xarray as xr
import numpy as np



In [2]:
from aurora import Aurora, Batch, Metadata
from aurora import Batch as BaseBatch
from aurora import Metadata as BaseMetadata
from aurora.batch import interpolate

# Import dependencies
import contextlib
from functools import partial
import dataclasses
from datetime import datetime, timedelta
import torch
from torch.nn import L1Loss  # = MAE: Mean Absolute Error = '.abs().mean()'
from torch.nn import Module  # for custom WeightedMAELoss (Aurora loss)
import torch.nn.functional as F  # for downsample batch
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from peft import get_peft_model
import tempfile
import requests
import pickle
import dask
import numpy as np
import pandas as pd
import xarray as xr
import zarr
import gcsfs
from typing import Union, Callable

In [3]:
# Aurora core
from aurora import Aurora, Batch, Metadata
from aurora.batch import interpolate

# Dataset & dataloader
from torch.utils.data import Dataset, DataLoader

# Model training utils
import torch
import torch.nn.functional as F
from torch.nn import L1Loss, Module
import torch.optim as optim
from torch.amp import autocast


# LoRA PEFT
from peft import get_peft_model, LoraConfig

# Data processing
import xarray as xr
import numpy as np
import pandas as pd

# Filesystem + storage
import gcsfs
import zarr

# General utilities
from typing import Union, Callable
from datetime import datetime, timedelta
import dataclasses
import tempfile
import contextlib
from functools import partial


# Avoid CUDA error: invalid configuration argument on F.scaled_dot_product_attention
# https://stackoverflow.com/questions/77343471/pytorch-cuda-error-invalid-configuration-argument
torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)
torch.backends.cuda.enable_math_sdp(True)


# Constants
STATIC_VARS_HF_URL = "https://huggingface.co/microsoft/aurora/resolve/main/aurora-0.25-static.pickle"
GCS_URL = "gs://weatherbench2/datasets/era5/1959-2023_01_10-wb13-6h-1440x721.zarr/"
LEAD_TIME = pd.Timedelta("6h")
AURORA_PRESSURE_LEVELS = [50, 100, 150, 200, 250, 300, 400, 500, 600, 700, 850, 925, 1000]
AURORA_VARIABLE_NAMES = {
    "surface": [
        '10u',
        '10v',
        '2t',
        'msl',
    ],
    "atmospheric": [
        't',
        'u',
        'v',
        'q',
        'z',
    ],
    "static": [
        'z',  # geopotential_at_sea_level
        'lsm',  # land_sea_mask
        'slt',  # soil_type
    ]
}
VARIABLES_STATISTICS: dict[str, tuple[float, float]] = { # mean, std (location, scale)
    'z': (-1386.496, 58844.67),
    'lsm': (0.0, 1.0),
    'slt': (0.0, 7.0),
    '2t': (278.514, 21.22036),
    '10u': (-0.05135059, 5.547512),
    '10v': (0.189158, 4.765339),
    'msl': (100957.8, 1332.246),
    'z_50': (199373.0, 5875.553),
    'z_100': (157642.1, 5510.64),
    'z_150': (133141.4, 5823.912),
    'z_200': (115330.0, 5820.169),
    'z_250': (101223.1, 5536.585),
    'z_300': (89414.15, 5091.916),
    'z_400': (69980.38, 4150.851),
    'z_500': (54115.37, 3353.187),
    'z_600': (40648.33, 2695.808),
    'z_700': (28928.82, 2136.436),
    'z_850': (13749.78, 1470.321),
    'z_925': (7015.005, 1228.997),
    'z_1000': (738.1545, 1072.307),
    'u_50': (5.653076, 15.29281),
    'u_100': (10.27951, 13.52611),
    'u_150': (13.54061, 16.04335),
    'u_200': (14.20915, 17.6763),
    'u_250': (13.34584, 17.9671),
    'u_300': (11.80173, 17.11917),
    'u_400': (8.817291, 14.34276),
    'u_500': (6.563273, 11.98419),
    'u_600': (4.814521, 10.33421),
    'u_700': (3.345237, 9.168821),
    'u_850': (1.418379, 8.188043),
    'u_925': (0.6172657, 7.940808),
    'u_1000': (-0.03328723, 6.141778),
    'v_50': (0.004226111, 7.058931),
    'v_100': (0.01411897, 7.47931),
    'v_150': (-0.03697671, 9.57199),
    'v_200': (-0.04507801, 11.88069),
    'v_250': (-0.02980338, 13.38039),
    'v_300': (-0.0229477, 13.34044),
    'v_400': (-0.01771003, 11.22955),
    'v_500': (-0.02387986, 9.181708),
    'v_600': (-0.02716674, 7.803569),
    'v_700': (0.02153583, 6.87104),
    'v_850': (0.142815, 6.264443),
    'v_925': (0.205348, 6.470644),
    'v_1000': (0.1867637, 5.308203),
    't_50': (212.4864, 10.26284),
    't_100': (208.4042, 12.52901),
    't_150': (213.3201, 8.928709),
    't_200': (218.0615, 7.189547),
    't_250': (222.771, 8.529282),
    't_300': (228.8696, 10.71679),
    't_400': (242.1368, 12.69102),
    't_500': (252.9492, 13.06447),
    't_600': (261.1347, 13.42046),
    't_700': (267.401, 14.76523),
    't_850': (274.56, 15.5888),
    't_925': (277.3572, 16.08798),
    't_1000': (281.013, 17.13983),
    'q_50': (2.67818e-06, 3.571687e-07),
    'q_100': (2.633677e-06, 5.703754e-07),
    'q_150': (5.254625e-06, 3.794077e-06),
    'q_200': (1.940632e-05, 2.267534e-05),
    'q_250': (5.773618e-05, 7.446644e-05),
    'q_300': (0.0001273861, 0.0001684361),
    'q_400': (0.0003855659, 0.0005078644),
    'q_500': (0.0008529599, 0.001079294),
    'q_600': (0.001541429, 0.001769722),
    'q_700': (0.002431637, 0.002549169),
    'q_850': (0.004575618, 0.004112368),
    'q_925': (0.006033134, 0.005071058),
    'q_1000': (0.007030342, 0.005913548)
}

class Xaurora(Aurora):
    def __init__(self, *args, **kwargs) -> None:
        self.autocast = kwargs.pop("autocast", True)  # Remove 'autocast' from kwargs
        self.autocast_dtype = torch.bfloat16

        super().__init__(*args, **kwargs)
        
    def forward(self, batch: Batch, lead_time: timedelta) -> Batch:
        member_id = batch.metadata.member_id

        # Move batch to correct dtype and device
        p = next(self.parameters())
        batch = batch.type(p.dtype)
        batch = batch.normalise()
        batch = batch.crop(patch_size=self.encoder.patch_size)
        batch = batch.to(p.device)

        # Determine resolution of latent patch representation
        H, W = batch.spatial_shape
        patch_res = (
            self.encoder.latent_levels,
            H // self.encoder.patch_size,
            W // self.encoder.patch_size,
        )

        # Repeat static variables to match batch/time dims
        B, T = next(iter(batch.surf_vars.values())).shape[:2]
        batch = dataclasses.replace(
            batch,
            static_vars={k: v[None, None].repeat(B, T, 1, 1) for k, v in batch.static_vars.items()},
        )

        # Encode inputs
        x = self.encoder(batch, lead_time=lead_time)

        # Apply backbone with autocast if enabled
        with torch.autocast(device_type="cuda", dtype=self.autocast_dtype) if self.autocast else contextlib.nullcontext():
            x = self.backbone(
                x,
                lead_time=lead_time,
                patch_res=patch_res,
                rollout_step=batch.metadata.rollout_step,
            )

        # Decode forecast
        pred = self.decoder(
            x,
            batch,
            lead_time=lead_time,
            patch_res=patch_res,
        )

        # Wrap in Batch object and restore metadata
        pred = Batch.from_aurora_batch(pred, member_id=member_id)
        assert pred.metadata.member_id == batch.metadata.member_id, "Member ID mismatch."

        # Remove temporal dims from static variables
        pred = dataclasses.replace(
            pred,
            static_vars={k: v[0, 0] for k, v in batch.static_vars.items()},
        )

        # Ensure output shape includes time dim (1 step)
        pred = dataclasses.replace(
            pred,
            surf_vars={k: v[:, None] for k, v in pred.surf_vars.items()},
            atmos_vars={k: v[:, None] for k, v in pred.atmos_vars.items()},
            metadata=dataclasses.replace(
                pred.metadata,
                rollout_step=batch.metadata.rollout_step + 1,
            )
        )

        # Unnormalize the output
        pred = pred.unnormalise()
        return pred

    
XauroraSmall = partial(
    Xaurora,
    encoder_depths=(2, 6, 2),
    encoder_num_heads=(4, 8, 16),
    decoder_depths=(2, 6, 2),
    decoder_num_heads=(16, 8, 4),
    embed_dim=256,
    num_heads=8,
    use_lora=False,
)


@dataclasses.dataclass
class Metadata(BaseMetadata):
    """Metadata in a batch.

    Args:
        lat (:class:`torch.Tensor`): Latitudes.
        lon (:class:`torch.Tensor`): Longitudes.
        time (tuple[datetime, ...]): For every batch element, the time.
        atmos_levels (tuple[int | float, ...]): Pressure levels for the atmospheric variables in
            hPa.
        rollout_step (int, optional): How many roll-out steps were used to produce this prediction.
            If equal to `0`, which is the default, then this means that this is not a prediction,
            but actual data. This field is automatically populated by the model and used to use a
            separate LoRA for every roll-out step. Generally, you are safe to ignore this field.
        member_id (int, optional): The member ID of the ensemble member. It defaults to `0`.
    """

    lat: torch.Tensor
    lon: torch.Tensor
    time: tuple[datetime, ...]
    atmos_levels: tuple[int | float, ...]
    rollout_step: int = 0
    member_id: int|list[int] = None


@dataclasses.dataclass
class Batch:
    """A batch of data.

    Args:
        surf_vars (dict[str, :class:`torch.Tensor`]): Surface-level variables with shape
            `(b, t, h, w)`.
        static_vars (dict[str, :class:`torch.Tensor`]): Static variables with shape `(h, w)`.
        atmos_vars (dict[str, :class:`torch.Tensor`]): Atmospheric variables with shape
            `(b, t, c, h, w)`.
        metadata (:class:`Metadata`): Metadata associated to this batch.
    """

    surf_vars: dict[str, torch.Tensor]
    static_vars: dict[str, torch.Tensor]
    atmos_vars: dict[str, torch.Tensor]
    metadata: Metadata
    
    @property
    def spatial_shape(self) -> tuple[int, int]:
        """Get the spatial shape from an arbitrary surface-level variable."""
        return next(iter(self.surf_vars.values())).shape[-2:]
    
    def save(self, path: str) -> None:
        """Save the batch to a file."""
        torch.save(self, path)
    
    @staticmethod
    def load(path: str) -> "Batch":
        """Load a batch from a file."""
        return torch.load(path, weights_only=False)
    
    def fillna(self, how: str="spatial_conv", **how_kwargs):
        if how == "spatial_conv":
            return Batch(
                surf_vars={k: fillna_spatial_mean_conv(v, **how_kwargs) for k, v in self.surf_vars.items()},
                static_vars={k: fillna_spatial_mean_conv(v, **how_kwargs) for k, v in self.static_vars.items()},
                atmos_vars={k: fillna_spatial_mean_conv(v, **how_kwargs) for k, v in self.atmos_vars.items()},
                metadata=self.metadata,
            )
        else:
            raise ValueError(f"Unknown fillna method {how}.")
    
    def normalise(
        self, 
        stats: dict[str, tuple[float, float]]=VARIABLES_STATISTICS
    ) -> "Batch":
        assert all(
            k in stats.keys() for k in self.surf_vars.keys()
        ), "Not all surface variables have statistics."
        return Batch(
            surf_vars={
                k: normalise_surf_var(v, k, stats=stats) for k, v in self.surf_vars.items()
            },
            static_vars={
                k: normalise_surf_var(v, k, stats=stats) for k, v in self.static_vars.items()
            },
            atmos_vars={
                k: normalise_atmos_var(v, k, self.metadata.atmos_levels, stats=stats)
                for k, v in self.atmos_vars.items()
            },
            metadata=self.metadata,
        )
    
    def unnormalise(
        self, 
        stats: dict[str, tuple[float, float]]=VARIABLES_STATISTICS
    ) -> "Batch":
        assert all(
            k in stats.keys() for k in self.surf_vars.keys()
        ), "Not all surface variables have statistics."
        return Batch(
            surf_vars={
                k: unnormalise_surf_var(v, k, stats=stats) for k, v in self.surf_vars.items()
            },
            static_vars={
                k: unnormalise_surf_var(v, k, stats=stats) for k, v in self.static_vars.items()
            },
            atmos_vars={
                k: unnormalise_atmos_var(v, k, self.metadata.atmos_levels, stats=stats)
                for k, v in self.atmos_vars.items()
            },
            metadata=self.metadata,
        )
        
    def crop(self, patch_size: int) -> "Batch":
        """Crop the variables in the batch to patch size `patch_size`."""
        h, w = self.spatial_shape

        if w % patch_size != 0:
            raise ValueError("Width of the data must be a multiple of the patch size.")

        if h % patch_size == 0:
            return self
        
        elif h % patch_size == 1:
            return Batch(
                surf_vars={k: v[..., :-1, :] for k, v in self.surf_vars.items()},
                static_vars={k: v[..., :-1, :] for k, v in self.static_vars.items()},
                atmos_vars={k: v[..., :-1, :] for k, v in self.atmos_vars.items()},
                metadata=Metadata(
                    lat=self.metadata.lat[:-1],
                    lon=self.metadata.lon,
                    atmos_levels=self.metadata.atmos_levels,
                    time=self.metadata.time,
                    rollout_step=self.metadata.rollout_step,
                    member_id=self.metadata.member_id,
                ),
            )
        else:
            raise ValueError(
                f"There can at most be one latitude too many, "
                f"but there are {h % patch_size} too many."
            )
            
    def crop_right(self, n: int) -> "Batch":
        """Crop the rightmost `n` columns of the variables in the batch."""
        return Batch(
            surf_vars={k: v[..., :-n, :] for k, v in self.surf_vars.items()},
            static_vars={k: v[..., :-n, :] for k, v in self.static_vars.items()},
            atmos_vars={k: v[..., :-n, :] for k, v in self.atmos_vars.items()},
            metadata=Metadata(
                lat=self.metadata.lat[:-n],
                lon=self.metadata.lon,
                atmos_levels=self.metadata.atmos_levels,
                time=self.metadata.time,
                rollout_step=self.metadata.rollout_step,
                member_id=self.metadata.member_id,
            ),
        )

    def _fmap(self, f: Callable[[torch.Tensor], torch.Tensor]) -> "Batch":
        return Batch(
            surf_vars={k: f(v) for k, v in self.surf_vars.items()},
            static_vars={k: f(v) for k, v in self.static_vars.items()},
            atmos_vars={k: f(v) for k, v in self.atmos_vars.items()},
            metadata=Metadata(
                lat=f(self.metadata.lat),
                lon=f(self.metadata.lon),
                atmos_levels=self.metadata.atmos_levels,
                time=self.metadata.time,
                rollout_step=self.metadata.rollout_step,
                member_id=self.metadata.member_id,
            ),
        )

    def to(self, device: str | torch.device) -> "Batch":
        """Move the batch to another device."""
        return self._fmap(lambda x: x.to(device))

    def type(self, t: type) -> "Batch":
        """Convert everything to type `t`."""
        return self._fmap(lambda x: x.type(t))

    def regrid(self, res: float) -> "Batch":
        """Regrid the batch to a `res` degrees resolution.

        This results in `float32` data on the CPU.

        This function is not optimised for either speed or accuracy. Use at your own risk.
        """

        shape = (round(180 / res) + 1, round(360 / res))
        lat_new = torch.from_numpy(np.linspace(90, -90, shape[0]))
        lon_new = torch.from_numpy(np.linspace(0, 360, shape[1], endpoint=False))
        interpolate_res = partial(
            interpolate,
            lat=self.metadata.lat,
            lon=self.metadata.lon,
            lat_new=lat_new,
            lon_new=lon_new,
        )

        return Batch(
            surf_vars={k: interpolate_res(v) for k, v in self.surf_vars.items()},
            static_vars={k: interpolate_res(v) for k, v in self.static_vars.items()},
            atmos_vars={k: interpolate_res(v) for k, v in self.atmos_vars.items()},
            metadata=Metadata(
                lat=lat_new,
                lon=lon_new,
                atmos_levels=self.metadata.atmos_levels,
                time=self.metadata.time,
                rollout_step=self.metadata.rollout_step,
                member_id=self.metadata.member_id,
            ),
        )
        
    def downsample(self, factor: int) -> "Batch":
        """Downsample batch by factor using Batch.regrid.
        
        See notes about using it carefully in .regrid"""
        
        # detach and to cpu 
        if self.metadata.lat.device != torch.device('cpu'):
            out = self._fmap(lambda x: x.detach().cpu())
        else:
            out = self
        
        _, W = out.spatial_shape
        current_resolution = 360 / W
        new_resolution = current_resolution * factor
        
        return out.regrid(new_resolution)
    
    @classmethod        
    def from_aurora_batch(cls, aurora_batch: BaseBatch, **metadata_kwargs) -> "Batch":
        return cls(
            surf_vars=aurora_batch.surf_vars,
            static_vars=aurora_batch.static_vars,
            atmos_vars=aurora_batch.atmos_vars,
            metadata=Metadata(
                lat=aurora_batch.metadata.lat,
                lon=aurora_batch.metadata.lon,
                time=aurora_batch.metadata.time,
                atmos_levels=aurora_batch.metadata.atmos_levels,
                rollout_step=aurora_batch.metadata.rollout_step,
                **metadata_kwargs
            ),
        )


class WeightedMAELoss(Module):  # custom loss functions in torch should inherit from Module
    """
    The original loss function that the Aurora team used as their training objective for
    both pre-training and fine-tuning.
    """
    def __init__(
        self,
        gamma=2.0,  # dataset weight
        alpha=0.25,  # overall surface loss weight
        beta=1.0,  # overall atmospheric loss weight
        surf_var_weights={},  # to store weights for individual variable weighting
        atmos_var_weights={},  # to store weights for individual variable weighting
    ):
        super().__init__()
        self.gamma = gamma
        self.alpha = alpha
        self.beta = beta
        self.l1loss = L1Loss(reduction='sum')  # sum of AE for individual variables that we'll normalize

        # Set default variable weights if not provided
        self.surf_var_weights = surf_var_weights if surf_var_weights is not None else {
            "2t": 1.0, "10u": 1.0, "10v": 1.0, "msl": 1.0
        }
        self.atmos_var_weights = atmos_var_weights if atmos_var_weights is not None else {
            "z": 1.0, "q": 1.0, "t": 1.0, "u": 1.0, "v": 1.0
        }

    def forward(self, pred_batch: Batch, target_batch: Batch) -> torch.Tensor:
        """
        Main method to compute the loss.

        Gets spatial dimensions (H, W) from predictions, assuming:
        - pred_batch.surf_vars has shape (batch, time, H, W)
        - atmos_vars has shape (batch, time, C, H, W)
        - time dimension for targets is 1.
        - H and W are consistent.

        pred_surf_shape example: (B, T, H, W) where T for pred is 1
        pred_atmos_shape example: (B, T, C, H, W) where T for pred is 1
        """
        # Get dimensions from the *predicted* batch, as target batch is constructed based on it
        # next(iter(...)) is used to get the first variable's tensor
        _, _, H, W = next(iter(pred_batch.surf_vars.values())).shape

        # Determine number of surface and atmospheric variables
        VS = len(pred_batch.surf_vars)
        VA = len(pred_batch.atmos_vars)

        # Use a tensor from the pred_batch to infer the correct device
        device = pred_batch.surf_vars[list(pred_batch.surf_vars.keys())[0]].device

        # Initialize total loss components
        surface_loss_sum = torch.tensor(0.0, device=device)
        atmospheric_loss_sum = torch.tensor(0.0, device=device)

        # Calculate Surface Loss Component.
        for k in pred_batch.surf_vars:
            # Get the variable-specific weight w^S_k
            w_S_k = self.surf_var_weights.get(k, 1.0)  # default to 1.0 if not found

            # L1Loss with reduction='sum' sums up all absolute differences for the variable
            # Then we normalize by H*W
            var_mae = self.l1loss(pred_batch.surf_vars[k], target_batch.surf_vars[k]) / (H * W)
            surface_loss_sum += w_S_k * var_mae

        # Calculate Atmospheric Loss Component
        for k in pred_batch.atmos_vars:
            # Get the variable-specific weight w^A_k (applied across all levels)
            w_A_k = self.atmos_var_weights.get(k, 1.0)  # default to 1.0 if not found

            # Get number of atmospheric levels (C) for this variable
            # Assuming pred_batch.atmos_vars[k] has shape (batch, time, C, H, W)
            _, _, C, _, _ = pred_batch.atmos_vars[k].shape

            # L1Loss with reduction='sum' sums up all absolute differences for the variable across all levels
            # Then we normalize by C*H*W
            var_mae = self.l1loss(pred_batch.atmos_vars[k], target_batch.atmos_vars[k]) / (C * H * W)
            atmospheric_loss_sum += w_A_k * var_mae

        # Combine weighted surface and atmospheric losses
        combined_loss_inside_brackets = self.alpha * surface_loss_sum + self.beta * atmospheric_loss_sum

        # Apply outer scaling factor
        final_loss = (self.gamma / (VS + VA)) * combined_loss_inside_brackets

        return final_loss


class AuroraDataset(Dataset):
    def __init__(
        self,
        # xr Datasets
        surface_ds: xr.Dataset,
        atmospheric_ds: xr.Dataset,
        static_ds: xr.Dataset,
        # variables
        surface_variables: list[str]=AURORA_VARIABLE_NAMES["surface"],
        atmospheric_variables: list[str]=AURORA_VARIABLE_NAMES["atmospheric"],
        static_variables: list[str]=AURORA_VARIABLE_NAMES["static"],
        # temporal parameters
        base_frequency: Union[str, pd.Timedelta]="6h",
        input_temporal_length: Union[str, int, pd.Timedelta]="12h",
        output_temporal_length: Union[str, int, pd.Timedelta]="6h",
        inter_sample_gap: Union[str, int, pd.Timedelta]="6h",
        forecast_horizon: Union[str, int, pd.Timedelta]="6w",
        init_frequency: Union[str, int, pd.Timedelta]="1d",
        init_gap: Union[str, int, pd.Timedelta]="0h",
        downsampling_rate: int=1,
    ):
        """
        Initialise the AuroraDataset.

        Args:
            surface_ds: xr.Dataset
                The surface dataset.
            atmospheric_ds: xr.Dataset
                The atmospheric dataset.
            static_ds: xr.Dataset
                The static dataset.
            surface_variables: list[str]
                The surface variables to include. Defaults to Aurora's surface variables.
            atmospheric_variables: list[str]
                The atmospheric variables to include. Defaults to Aurora's atmospheric variables.
            static_variables: list[str]
                The static variables to include. Defaults to Aurora's static variables.
            base_frequency: Union[str, pd.Timedelta]
                The base frequency of the data. Defaults to 6 hours.
            input_temporal_length: Union[str, int, pd.Timedelta]
                The length of the input time series. If `int`, considered to be a multiple of `base_frequency`. Defaults to 12 hours.
            output_temporal_length: Union[str, int, pd.Timedelta]
                The length of the output time series. If `int`, considered to be a multiple of `base_frequency`. Defaults to 6 hours.
            inter_sample_gap: Union[str, int, pd.Timedelta]
                The gap between consecutive samples. If `int`, considered to be a multiple of `base_frequency`. Defaults to 6 hours.
            forecast_horizon: Union[str, int, pd.Timedelta]
                The forecast horizon. If `int`, considered to be a multiple of `base_frequency`. Defaults to 6 weeks.
            init_frequency: Union[str, int, pd.Timedelta]
                The frequency of the initialisation times. If `int`, considered to be a multiple of `base_frequency`. Defaults to 1 day.
            init_gap: Union[str, int, pd.Timedelta]
                The gap between the initialisation time and the start of the input time series. If `int`, considered to be a multiple of 
                `base_frequency`. Defaults to 0 hours.
            downsampling_rate: int
                The downsampling rate of the dataset. Defaults to 1.
        """
        super().__init__()

        # get sorted datasets
        # Aurora requires latitudes to be in descending order
        # and longitudes to be in ascending order
        self.surface_ds = surface_ds.sortby("latitude", ascending=False).sortby("longitude", ascending=True)
        self.atmospheric_ds = atmospheric_ds.sortby("latitude", ascending=False).sortby("longitude", ascending=True)
        self.static_ds = static_ds.sortby("latitude", ascending=False).sortby("longitude", ascending=True)

        self.atmospheric_variables = atmospheric_variables
        self.surface_variables = surface_variables
        self.static_variables = static_variables

        self.base_frequency = pd.Timedelta(base_frequency) if isinstance(base_frequency, str) else base_frequency
        self.input_temporal_length = convert_to_steps(input_temporal_length, self.base_frequency, check_valid=True)
        self.output_temporal_length = convert_to_steps(output_temporal_length, self.base_frequency, check_valid=True)
        self.inter_sample_gap = convert_to_steps(inter_sample_gap, self.base_frequency, check_valid=True)
        self.forecast_horizon = convert_to_steps(forecast_horizon, self.base_frequency, check_valid=True)
        self.init_frequency = convert_to_steps(init_frequency, self.base_frequency, check_valid=True)
        self.init_gap = convert_to_steps(init_gap, self.base_frequency, check_valid=True)

        self.downsampling_rate = downsampling_rate
        self._spatial_shape = None

        # extract timestamps
        assert (surface_ds.time == atmospheric_ds.time).all(), f"got different timestamps for surface and atmospheric data."
        self.timestamps = self.surface_ds.time.values.astype("datetime64[s]")
        
        # add init gap
        if self.init_gap < 0:
            # reverse the init gap index based on the length of self.timestamps
            self.init_gap = len(self.timestamps) + self.init_gap
        self.timestamps = self.timestamps[self.init_gap:]        

        # compute feasible length of the time series
        discarded_timesteps = self.forecast_horizon + self.output_temporal_length + self.input_temporal_length + self.inter_sample_gap
        self.num_samples = len(self.timestamps) - discarded_timesteps
        assert self.timestamps[self.num_samples-1] + discarded_timesteps * self.base_frequency == self.timestamps[-1]

        # add init frequency
        self.num_samples = self.num_samples // self.init_frequency


    @classmethod
    def from_cloud_storage(
        cls,
        gcs_url: str,
        start_year: str|int=None,
        end_year: str|int=None,
        static_url: str=STATIC_VARS_HF_URL,
        atmospheric_variables: list[str]=AURORA_VARIABLE_NAMES["atmospheric"],
        surface_variables: list[str]=AURORA_VARIABLE_NAMES["surface"],
        pressure_levels: list[int]=AURORA_PRESSURE_LEVELS,
        variable_names_map: dict[str, str]=None,
        **kwargs
    ) -> "AuroraDataset":
        """
        No need to download anything locally!

        Args:
            gcs_url: str
                The url to the zarr store in the cloud storage.
            static_url: str
                The url to the static dataset in the cloud storage.
            atmospheric_variables: list[str]
                The atmospheric variables to include. Defaults to Aurora's atmospheric variables.
            surface_variables: list[str]
                The surface variables to include. Defaults to Aurora's surface variables.
            pressure_levels: list[int]
                The pressure levels to include. Defaults to Aurora's pressure levels.
            variable_names_map: dict[str, str]
                A dictionary mapping the variable names to the desired names.
            **kwargs:
                Additional arguments to pass to the AuroraDataset constructor.
        Returns:
            AuroraDataset
        """
        static_ds = load_static_ds_local("static_data/static.nc")

        # load surface and atmospheric ds
        surface_ds, atmospheric_ds = load_gcs_datasets(
            gcs_url=gcs_url,
            start_year=start_year,
            end_year=end_year,
            atmospheric_variables=atmospheric_variables,
            surface_variables=surface_variables,
            pressure_levels=pressure_levels,
            variable_names_map=variable_names_map
        )
        return cls(
            surface_ds=surface_ds,
            atmospheric_ds=atmospheric_ds,
            static_ds=static_ds,
            **kwargs
        )

    @property
    def spatial_shape(self) -> tuple[int]:
        """
        return (nlat, nlon)
        """
        # return if already computed
        if self._spatial_shape is not None:
            return self._spatial_shape
        # compute
        batch = self[0][0]
        if self.downsampling_rate is not None and self.downsampling_rate > 1:
            with torch.no_grad():
                batch = downsample_batch(batch, self.downsampling_rate)

        _,_,H,W = batch.surf_vars[next(iter(batch.surf_vars.keys()))].shape
        self._spatial_shape = (H, W)
        return self._spatial_shape

    def get_matching_timestamp(self, timestamp: np.datetime64|datetime) -> np.datetime64:
        """
        Find a create an OUTPUT batch from the requested timestamp. 
        """
        if self.output_temporal_length > 1:
            raise NotImplementedError("get_matching_timestamp is only implemented for output_temporal_length=1.")

        if isinstance(timestamp, datetime):
            timestamp = np.datetime64(timestamp)

        # get output slice
        output_slice = [
            timestamp + np.timedelta64(pd.Timedelta(self.base_frequency).value * n, "s")
            for n in range(self.output_temporal_length)
        ]

        # return the batch
        return xr_to_batch(
            self.surface_ds.sel(time=output_slice).compute(),
            self.atmospheric_ds.sel(time=output_slice).compute(),
            self.static_ds.compute(),
            surface_variables=self.surface_variables,
            static_variables=self.static_variables,
            atmospheric_variables=self.atmospheric_variables
        )

    def get_matching_output(self, batch: Batch) -> Batch:
        """
        Creates a Batch from the requested timestamp. The timestamp is the first 
        timestamp of the output time series of length `output_temporal_length`.
        """
        T = batch.atmos_vars[next(iter(batch.atmos_vars.keys()))].shape[1]
        if T != 1:
            raise NotImplementedError("get_matching_output is only implemented for T=1.")

        # get the times
        times = list(batch.metadata.time)
        output_batch = []

        # loop over requested times
        for time in times:
            new = self.get_matching_timestamp(time)
            output_batch.append(new)

        # collate
        output_batch = batch_collate_fn(output_batch)

        # apply downsampling
        if self.downsampling_rate is not None and self.downsampling_rate > 1:
            with torch.no_grad():
                output_batch = downsample_batch(output_batch, self.downsampling_rate)

        return output_batch

    def __len__(self) -> int:
        return self.num_samples

    def __getitem__(self, idx: int) -> Batch:
        if idx < 0:
            idx = self.num_samples + idx

        # add init frequency
        idx = idx * self.init_frequency

        # get input and output indexes
        input_slice = slice(idx, idx+self.input_temporal_length)
        output_slice = slice(idx+self.input_temporal_length+self.inter_sample_gap-1,
                             idx+self.input_temporal_length+self.inter_sample_gap-1+self.output_temporal_length)

        # get timestamps
        input_timestamps = self.timestamps[input_slice]
        output_timestamps = self.timestamps[output_slice]

        # make a few checks
        assert len(input_timestamps) == self.input_temporal_length
        assert len(output_timestamps) == self.output_temporal_length
        assert input_timestamps[-1] + self.inter_sample_gap * pd.Timedelta(self.base_frequency) == output_timestamps[0]

        input_batch = xr_to_batch(
            self.surface_ds.sel(time=input_timestamps).compute(),
            self.atmospheric_ds.sel(time=input_timestamps).compute(),
            self.static_ds.compute(),
            surface_variables=self.surface_variables,
            static_variables=self.static_variables,
            atmospheric_variables=self.atmospheric_variables
        )

        output_batch = xr_to_batch(
            self.surface_ds.sel(time=output_timestamps).compute(),
            self.atmospheric_ds.sel(time=output_timestamps).compute(),
            self.static_ds.compute(),
            surface_variables=self.surface_variables,
            static_variables=self.static_variables,
            atmospheric_variables=self.atmospheric_variables
        )

        if self.downsampling_rate is not None and self.downsampling_rate > 1:
            with torch.no_grad():
                input_batch = downsample_batch(input_batch, self.downsampling_rate)
                output_batch = downsample_batch(output_batch, self.downsampling_rate)

        return input_batch, output_batch


# Functions
def normalise_surf_var(
    x: torch.Tensor,
    name: str,
    stats: dict[str, tuple[float, float]],
    unnormalise: bool = False,
) -> torch.Tensor:
    """Normalise a surface-level variable."""
    location, scale = stats[name]
    if unnormalise:
        return x * scale + location
    else:
        return (x - location) / scale


def normalise_atmos_var(
    x: torch.Tensor,
    name: str,
    atmos_levels: tuple[int | float, ...],
    stats: dict[str, tuple[float, float]],
    unnormalise: bool = False,
) -> torch.Tensor:
    """Normalise an atmospheric variable."""
    level_locations: list[int | float] = []
    level_scales: list[int | float] = []
    for level in atmos_levels:
        name_level = f"{name}_{level}"
        level_locations.append(stats[name_level][0])
        level_scales.append(stats[name_level][1])
    location = torch.tensor(level_locations, dtype=x.dtype, device=x.device)
    scale = torch.tensor(level_scales, dtype=x.dtype, device=x.device)

    if unnormalise:
        return x * scale[..., None, None] + location[..., None, None]
    else:
        return (x - location[..., None, None]) / scale[..., None, None]


unnormalise_surf_var = partial(normalise_surf_var, unnormalise=True)
unnormalise_atmos_var = partial(normalise_atmos_var, unnormalise=True)


def fillna_spatial_mean_conv(tensor: torch.Tensor, kernel_size: int = 3, iterations: int = 5) -> torch.Tensor:
    """
    Approximate filling of NaNs via convolutional local mean.
    
    Supports:
      - 4D (B, T, H, W)
      - 5D (B, T, L, H, W)

    Args:
        tensor: Tensor with NaNs (on GPU or CPU).
        kernel_size: Spatial window size (must be odd).
        iterations: Number of fill iterations.

    Returns:
        Tensor with NaNs replaced by local mean.
    """
    assert kernel_size % 2 == 1, "kernel_size must be odd"
    
    if not torch.isnan(tensor).any():
        return tensor  # No NaNs — early exit

    device = tensor.device
    filled = tensor.clone()
    nan_mask = torch.isnan(filled)
    filled[nan_mask] = 0  # Replace NaNs with 0s for now

    # Set up convolution kernel
    kernel_dim = 2 if tensor.ndim == 4 else 3
    spatial_dims = (-2, -1) if kernel_dim == 2 else (-3, -2, -1)
    padding = kernel_size // 2

    # Define kernel
    kernel_shape = [1] * (tensor.ndim - kernel_dim) + [1] * kernel_dim
    kernel = torch.ones(kernel_shape[:-kernel_dim] + [kernel_size] * kernel_dim, device=device)

    for _ in range(iterations):
        valid_mask = ~torch.isnan(tensor)
        weights = valid_mask.float()
        values = filled * weights

        if kernel_dim == 2:
            smoothed_vals = F.conv2d(values.view(-1, 1, *values.shape[-2:]), kernel, padding=padding, groups=1)
            smoothed_weights = F.conv2d(weights.view(-1, 1, *weights.shape[-2:]), kernel, padding=padding, groups=1)
        else:  # 3D
            smoothed_vals = F.conv3d(values.view(-1, 1, *values.shape[-3:]), kernel, padding=padding, groups=1)
            smoothed_weights = F.conv3d(weights.view(-1, 1, *weights.shape[-3:]), kernel, padding=padding, groups=1)

        smoothed_mean = smoothed_vals / (smoothed_weights + 1e-6)
        smoothed_mean = smoothed_mean.view_as(filled)

        # Update only NaN locations
        filled[nan_mask] = smoothed_mean[nan_mask]

    return filled


def convert_to_steps(temporal_length: Union[str, int, pd.Timedelta], 
                      base_frequency: Union[str, pd.Timedelta],
                      check_valid: bool=True) -> int:
    # convert to timedelta
    if isinstance(temporal_length, str):
        temporal_length = pd.Timedelta(temporal_length)
    elif isinstance(temporal_length, int):
        return temporal_length
    # check if it is a multiple of the base frequency
    if check_valid:
        assert temporal_length % pd.Timedelta(base_frequency) == pd.Timedelta(0), \
            "temporal_length must be a multiple of base_frequency"
    return int(temporal_length / base_frequency)


def downsample_batch(batch, factor, pooling=F.avg_pool2d):
    return batch.downsample(factor)


def batch_collate_fn(batches: list[Batch]|None) -> Batch:
    """
    Custom collate function to combine multiple Batch objects. Collating custom
    objects such as Batch in PyTorch requires a custom function. For efficient
    parallel processing on the GPU, individual samples are combined into a single
    larger 'batch'. In the context of Aurora, one 'sample' is one Aurora Batch
    instance: all the atmospheric and surface variables for a single point in
    time at a specific location.
    """
    # Check whether the input is of type Batch
    _batches = batches.copy()
    for i, batch in enumerate(_batches):
        if batch is None: batches.pop(i)
        elif not isinstance(batch, Batch):
            raise ValueError(f"Expected a list of Aurora batches or NoneType, got {type(batch)}")
    batches = _batches

    if len(batches) == 0:
        return # nothing to batch return None
    elif len(batches) == 1:
        return batches[0] # nothing to batch return the single batch

    # Prediction batches have a single time sample apparently
    times = []
    for batch in batches:
        time = batch.metadata.time
        if isinstance(time, (tuple, list)):
            times.extend(list(time))
        else:
            times.append(time)
    times = tuple(times)

    # batch the data
    return Batch(
        surf_vars={
            var: torch.cat([batch.surf_vars[var] for batch in batches], dim=0)
            for var in batches[0].surf_vars
        },
        atmos_vars={
            var: torch.cat([batch.atmos_vars[var] for batch in batches], dim=0)
            for var in batches[0].atmos_vars
        },
        static_vars={
            var: batches[0].static_vars[var]
            for var in batches[0].static_vars
        },
        metadata=Metadata(
            lat=batches[0].metadata.lat,
            lon=batches[0].metadata.lon,
            atmos_levels=batches[0].metadata.atmos_levels,
            rollout_step=batches[0].metadata.rollout_step,
            time=times,
        )
    )


def batch_pairs_collate_fn(batch_pairs: list[tuple[Batch, Batch]]) -> tuple[Batch, Batch]:
    """
    Collate function for batch input/output pairs. This function is a wrapper designed to 
    handle the common scenario in supervised learning where the DataLoader yields pairs 
    of (input, target).

    Input: It expects a list where each element is a tuple containing an input Batch and a 
    target Batch. 
    E.g., [(input_batch_1, target_batch_1), (input_batch_2, target_batch_2), ...] could be 
    such a list.
    """
    input_batch = batch_collate_fn([batch_pair[0] for batch_pair in batch_pairs])
    output_batch = batch_collate_fn([batch_pair[1] for batch_pair in batch_pairs])

    return input_batch, output_batch


def prepare_lons_lats(lons: np.ndarray, lats: np.ndarray) -> tuple[torch.Tensor, torch.Tensor]:
    """
    Aurora requires decreasing latitudes and increasing longitudes
    """
    if lats[0] < lats[1]: # i.e. increasing
        lats = torch.from_numpy(lats[::-1].copy())
        flip_lats = True
    else:
        lats = torch.from_numpy(lats)
        flip_lats = False

    if lons[0] < lons[1]: # i.e. increasing
        lons = torch.from_numpy(lons)
        flip_lons = False
    else:
        lons = torch.from_numpy(lons[::-1].copy())
        flip_lons = True

    return lons, lats, flip_lats, flip_lons


def prepare_array(x: np.ndarray, shape: tuple[int], flip_lons: bool, flip_lats: bool) -> torch.Tensor:
    x = x.reshape(shape).copy()
    if flip_lons: x = x[...,::-1].copy()
    if flip_lats: x = x[...,::-1,:].copy()
    return torch.from_numpy(x).clone()


def rename_xr_variables(ds: xr.Dataset, variable_names_map: dict[str, str]):
    """
    Directly taken from Eliot's data/utils.py.
    """
    # Intersect with data_vars
    variable_names_map = {k: v for k, v in variable_names_map.items() if k in list(ds.data_vars)+list(ds.coords)}

    # Rename the variables in the dataset
    renamed_ds = ds.rename(variable_names_map)

    return renamed_ds


def load_gcs_datasets(
    gcs_url: str=GCS_URL,
    num_debug_timesteps: int=0,
    start_year: str|int=None,
    end_year: str|int=None,
    atmospheric_variables: list[str]=AURORA_VARIABLE_NAMES["atmospheric"],
    surface_variables: list[str]=AURORA_VARIABLE_NAMES["surface"],
    pressure_levels: list[int]=AURORA_PRESSURE_LEVELS,
    variable_names_map: dict[str, str]=None,
):
    """
    A modified version of the function in Eliot's data/utils.py, allowing for
    selecting only a small number of recent time points for debugging
    purposes.
    """
    if start_year is None:
        assert end_year is None, "If start_year is None, end_year must be None"
    if end_year is None:
        assert start_year is None, "If end_year is None, start_year must be None"

    ds = xr.open_dataset(
        gcs_url,
        engine="zarr",
        chunks={},
        storage_options={"token": "anon"},
    )

    if num_debug_timesteps:  # Debug mode: select a small number of recent time points
        ds = ds.isel(time=slice(-num_debug_timesteps, None))

        # Create a slice from the selected times
        time_slice = slice(ds.time.values[0], ds.time.values[-1])
    elif start_year is not None and end_year is not None:
        start_date = f"{start_year}-01-01"
        end_date = f"{end_year}-12-31"
        time_slice = slice(start_date, end_date)
    else:
        time_slice = None

    # Remove -90 from latitude (i.e. 721 -> 720)
    # Solves ValueError: cannot reshape array of size 1036800 into shape (721,1440)
    if 90 in ds.latitude.values and -90 in ds.latitude.values:
        ds = ds.where(ds.latitude!=-90, drop=True)

    # Get the rename map
    if variable_names_map is None:
        freq = pd.infer_freq(ds.time.values)

        if not freq[0].isnumeric():
            freq = "1" + freq

        if pd.Timedelta(freq) in [pd.Timedelta("6h"), pd.Timedelta("24h")]:
            variable_names_map = ERA5_HRES_T0_WB2_VARIABLE_NAMES_MAP
        else:
            raise ValueError(f"No variable name map for freq {freq}")

    if len(surface_variables) > 0:
        surface_ds = rename_xr_variables(
            ds,
            variable_names_map
        )[surface_variables]

        if time_slice is not None:
            surface_ds = surface_ds.sel(time=time_slice)
            surface_ds = surface_ds.sortby("latitude", ascending=False).sortby("longitude", ascending=True)
    else:
        surface_ds = None

    if len(atmospheric_variables) > 0:
        atmospheric_ds = rename_xr_variables(
            ds,
            variable_names_map
        )[atmospheric_variables].sel(level=pressure_levels)

        if time_slice is not None:
            atmospheric_ds = atmospheric_ds.sel(time=time_slice)
            atmospheric_ds = atmospheric_ds.sortby("latitude", ascending=False).sortby("longitude", ascending=True)
    else:
        atmospheric_ds = None

    return surface_ds, atmospheric_ds


def xr_to_batch(
    surface_ds: xr.Dataset,
    atmospheric_ds: xr.Dataset,
    static_ds: xr.Dataset,
    surface_variables: list[str]=AURORA_VARIABLE_NAMES["surface"],
    static_variables: list[str]=AURORA_VARIABLE_NAMES["static"],
    atmospheric_variables: list[str]=AURORA_VARIABLE_NAMES["atmospheric"],
) -> Batch:
    """
    Create an Aurora Batch from XR Datasets.

    inspired by https://microsoft.github.io/aurora/example_era5.html
    and https://microsoft.github.io/aurora/example_hres_t0.html
    """
    # Converting to `datetime64[s]` ensures that the output of `tolist()` gives
    # `datetime.datetime`s. Note that this needs to be a tuple of length one:
    # one value for every batch element.
    # temporally, we want index 0 to be PREVIOUS time step and index 1 to be CURRENT time step
    # the metadata 'time' refers to the CURRENT time step, i.e. the last index
    if surface_ds.sizes["time"] == 1:
        _time = (surface_ds.time.values[0].astype("datetime64[s]").item(), )
    else:
        times = list(sorted(surface_ds.time.values.astype("datetime64[s]").tolist()))
        _time = (times[-1],) # only the last

    # get shapes for explicit reshaping and get lons, lats, levels
    # the process is repeated for each dataset because the
    # datasets can be empty
    if static_ds is not None and len(static_ds) > 0:
        H, W = static_ds.sizes["latitude"], static_ds.sizes["longitude"]

        # 1. get lons, lats
        lons = static_ds.longitude.values.copy()
        lats = static_ds.latitude.values.copy()

        # 2. prepare lons, lats
        lons, lats, flip_lats_static, flip_lons_static = prepare_lons_lats(lons, lats)

    else:
        flip_lats_static = False
        flip_lons_static = False

    if surface_ds is not None and len(surface_ds) > 0:
        # sort time
        surface_ds = surface_ds.sortby("time", ascending=True)
        T, H, W = surface_ds.sizes["time"], surface_ds.sizes["latitude"], surface_ds.sizes["longitude"]

        # 1. get lons, lats
        lons = surface_ds.longitude.values.copy()
        lats = surface_ds.latitude.values.copy()

        # 2. prepare lons, lats
        lons, lats, flip_lats_surface, flip_lons_surface = prepare_lons_lats(lons, lats)
    else:
        flip_lats_surface = False
        flip_lons_surface = False

    if atmospheric_ds is not None and len(atmospheric_ds) > 0:
        # sort time
        atmospheric_ds = atmospheric_ds.sortby("time", ascending=True)
        C, T, H, W = (
            atmospheric_ds.sizes["level"],
            atmospheric_ds.sizes["time"],
            atmospheric_ds.sizes["latitude"],
            atmospheric_ds.sizes["longitude"],
        )
        # 1. get lons, lats
        lons = atmospheric_ds.longitude.values.copy()
        lats = atmospheric_ds.latitude.values.copy()

        # 2. prepare lons, lats
        lons, lats, flip_lats_atmospheric, flip_lons_atmospheric = prepare_lons_lats(lons, lats)

        # 3. get levels
        levels = tuple(int(level) for level in atmospheric_ds.level.values)
    else:
        levels = tuple()
        flip_lats_atmospheric = False
        flip_lons_atmospheric = False

    return Batch.from_aurora_batch(
        BaseBatch(
            surf_vars = {
                var: prepare_array(
                    surface_ds[var].values,
                    (1, T, H, W),
                    flip_lats=flip_lats_surface,
                    flip_lons=flip_lons_surface)
                for var in surface_variables
            },
            atmos_vars = {
                var: prepare_array(
                    atmospheric_ds[var].values,
                    (1, T, C, H, W),
                    flip_lats=flip_lats_atmospheric,
                    flip_lons=flip_lons_atmospheric)
                for var in atmospheric_variables
            },
            static_vars = {
                var: prepare_array(
                    static_ds[var].values,
                    (H, W),
                    flip_lats=flip_lats_static,
                    flip_lons=flip_lons_static)
                for var in static_variables
            } if static_ds is not None and len(static_ds) > 0 else {},
            metadata=BaseMetadata(
                lat=lats,
                lon=lons,
                time=_time,
                atmos_levels=levels,
            )
        )
    )



def load_static_ds_local(path: str) -> xr.Dataset:
    ds = xr.open_dataset(path, engine="netcdf4")
    if 90 in ds.latitude.values and -90 in ds.latitude.values:
        ds = ds.where(ds.latitude!=-90, drop=True)
    return ds


# Modified version of 'ERA5_HRES_T0_WB2_VARIABLE_NAMES_MAP_6HR' in Eliot's data/constants.py
ERA5_HRES_T0_WB2_VARIABLE_NAMES_MAP = {
    # Surface-level Variables
    '10m_u_component_of_wind': '10u',
    '10m_v_component_of_wind': '10v',
    '2m_temperature': '2t',
    'mean_sea_level_pressure': 'msl',

    # Atmospheric Variables
    'temperature': 't',
    'u_component_of_wind': 'u',
    'v_component_of_wind': 'v',
    'specific_humidity': 'q',
    'geopotential': 'z',
}

# Initialize the base (pre-trained) Aurora model
base_model = XauroraSmall(use_lora=False, autocast=True)
base_model.load_checkpoint(
    "microsoft/aurora", 
    "aurora-0.25-small-pretrained.ckpt", 
    strict=False,  # to avoid error when loading state dict after disabling lora (finetuned model has lora weights)
)

names_target_modules = [
    "backbone.encoder_layers.0.blocks.0.attn.qkv",
    "backbone.encoder_layers.0.blocks.0.attn.proj",
    "backbone.encoder_layers.0.blocks.1.attn.qkv",
    "backbone.encoder_layers.0.blocks.1.attn.proj",
    "backbone.encoder_layers.1.blocks.0.attn.qkv",
    "backbone.encoder_layers.1.blocks.0.attn.proj",
    "backbone.encoder_layers.1.blocks.1.attn.qkv",
    "backbone.encoder_layers.1.blocks.1.attn.proj",
    "backbone.encoder_layers.1.blocks.2.attn.qkv",
    "backbone.encoder_layers.1.blocks.2.attn.proj",
    "backbone.encoder_layers.1.blocks.3.attn.qkv",
    "backbone.encoder_layers.1.blocks.3.attn.proj",
    "backbone.encoder_layers.1.blocks.4.attn.qkv",
    "backbone.encoder_layers.1.blocks.4.attn.proj",
    "backbone.encoder_layers.1.blocks.5.attn.qkv",
    "backbone.encoder_layers.1.blocks.5.attn.proj",
    "backbone.encoder_layers.2.blocks.0.attn.qkv",
    "backbone.encoder_layers.2.blocks.0.attn.proj",
    "backbone.encoder_layers.2.blocks.1.attn.qkv",
    "backbone.encoder_layers.2.blocks.1.attn.proj",
    "backbone.decoder_layers.0.blocks.0.attn.qkv",
    "backbone.decoder_layers.0.blocks.0.attn.proj",
    "backbone.decoder_layers.0.blocks.1.attn.qkv",
    "backbone.decoder_layers.0.blocks.1.attn.proj",
    "backbone.decoder_layers.1.blocks.0.attn.qkv",
    "backbone.decoder_layers.1.blocks.0.attn.proj",
    "backbone.decoder_layers.1.blocks.1.attn.qkv",
    "backbone.decoder_layers.1.blocks.1.attn.proj",
    "backbone.decoder_layers.1.blocks.2.attn.qkv",
    "backbone.decoder_layers.1.blocks.2.attn.proj",
    "backbone.decoder_layers.1.blocks.3.attn.qkv",
    "backbone.decoder_layers.1.blocks.3.attn.proj",
    "backbone.decoder_layers.1.blocks.4.attn.qkv",
    "backbone.decoder_layers.1.blocks.4.attn.proj",
    "backbone.decoder_layers.1.blocks.5.attn.qkv",
    "backbone.decoder_layers.1.blocks.5.attn.proj",
    "backbone.decoder_layers.2.blocks.0.attn.qkv",
    "backbone.decoder_layers.2.blocks.0.attn.proj",
    "backbone.decoder_layers.2.blocks.1.attn.qkv",
    "backbone.decoder_layers.2.blocks.1.attn.proj"
]



In [4]:
import dask
ds = xr.open_zarr(
    "gs://weatherbench2/datasets/era5/1959-2023_01_10-wb13-6h-1440x721.zarr/",
    consolidated=True,
    storage_options={"token": None}
)

train_ds = ds.sel(time=slice("2016-01-01", "2020-12-31"))
val_ds = ds.sel(time=slice("2021-01-01", "2022-01-01"))
test_ds = ds.sel(time=slice("2022-01-01", "2023-01-01"))


dataset = AuroraDataset.from_cloud_storage(
    gcs_url="gs://weatherbench2/datasets/era5/1959-2023_01_10-wb13-6h-1440x721.zarr/",  
    start_year=2016,
    end_year=2023
)

# Extract datasets for regional slicing
surface_ds = dataset.surface_ds
atmospheric_ds = dataset.atmospheric_ds
static_ds = dataset.static_ds
#For fine tuning do fro 2016-2020 including 2020

In [13]:
REGIONS = {
    "western_med": {
        "lat": slice(45.75, 32.0),
        "lon": slice(348.0, 376.0)  # -12 to 16 in 0–360 format
    },
    "eastern_med": {
        "lat": slice(43.75, 30.0),
        "lon": slice(16.0, 43.75)   # unchanged, direct match to east_coords
    }
}

def compute_climatology(ds: xr.Dataset, region: str, variable_names: list[str]) -> xr.Dataset:
    """
    Computes daily climatology (by day-of-year) for given variables in a region.
    Ensures consistent shape and coordinates for safe downstream evaluation.
    """
    print(f"Computing climatology for region: {region}...")
    # Slice region first
    ds_region = extract_region(ds, region)

    # Get day of year
    doy = ds_region['time'].dt.dayofyear
    print(f"  → Variables: {variable_names}")
    print(f"  → Time range: {str(ds_region.time.values[0])} to {str(ds_region.time.values[-1])}")
    print(f"  → Grid shape: {ds_region[variable_names[0]].isel(time=0).shape}")

    # Compute mean for each day-of-year
    clim = ds_region[variable_names].groupby(doy).mean('time', keep_attrs=True)

    # Preserve original coordinates to avoid extract_region later
    clim = clim.assign_coords(latitude=ds_region.latitude, longitude=ds_region.longitude)
    print("  → Climatology computed.\n")
    return clim


def extract_region(ds: xr.Dataset, region_name: str) -> xr.Dataset:
    region = REGIONS[region_name]
    lat_slice = region["lat"]
    lon_slice = region["lon"]

    if isinstance(lon_slice, list):  # wraparound
        return xr.concat(
            [ds.sel(latitude=lat_slice, longitude=ls) for ls in lon_slice],
            dim="longitude"
        )
    else:
        return ds.sel(latitude=lat_slice, longitude=lon_slice)
        
all_surface_vars = list(surface_ds.data_vars)
all_atmos_vars = list(atmospheric_ds.data_vars)

# Merge surface and atmospheric validation data
val_surf = surface_ds.sel(time=slice("2021-01-01", "2022-01-01"))
val_atmos = atmospheric_ds.sel(time=slice("2021-01-01", "2022-01-01"))
val_combined = xr.merge([val_surf, val_atmos])

# Compute climatology directly for region
climatology_ds = compute_climatology(val_combined, region, all_surface_vars + all_atmos_vars)



Computing climatology for region: western_med...
  → Variables: ['10u', '10v', '2t', 'msl', 't', 'u', 'v', 'q', 'z']
  → Time range: 2021-01-01T00:00:00.000000000 to 2022-01-01T18:00:00.000000000
  → Grid shape: (56, 48)
  → Climatology computed.



In [14]:
from copy import deepcopy
from sklearn.metrics import mean_squared_error
import numpy as np

# ------------------ UTILITY FUNCTIONS ------------------ #

def move_batch_to_cpu(batch):
    return Batch(
        surf_vars={k: v.cpu() for k, v in batch.surf_vars.items()},
        atmos_vars={k: v.cpu() for k, v in batch.atmos_vars.items()},
        static_vars={k: v.cpu() for k, v in batch.static_vars.items()},
        metadata=batch.metadata
    )

def roll_forward(input_batch: Batch, prediction: Batch) -> Batch:
    def roll_tensor(tensor_old, tensor_new):
        return torch.cat([tensor_old[:, 1:], tensor_new], dim=1)

    new_surf = {
        var: roll_tensor(input_batch.surf_vars[var], prediction.surf_vars[var][:, -1:])
        for var in input_batch.surf_vars
    }

    new_atmos = {
        var: roll_tensor(input_batch.atmos_vars[var], prediction.atmos_vars[var][:, -1:])
        for var in input_batch.atmos_vars
    }

    new_metadata = Metadata(
        lat=input_batch.metadata.lat,
        lon=input_batch.metadata.lon,
        time=(prediction.metadata.time[-1],),
        atmos_levels=input_batch.metadata.atmos_levels,
    )

    return Batch(
        surf_vars=new_surf,
        atmos_vars=new_atmos,
        static_vars=input_batch.static_vars,
        metadata=new_metadata
    )

def get_start_batch(surface_ds, atmospheric_ds, static_ds, region: str, time: datetime) -> Batch:
    time_window = [time - timedelta(hours=6), time]

    surf_slice = surface_ds.sel(time=time_window, method="nearest")
    atmos_slice = atmospheric_ds.sel(time=time_window, method="nearest")

    surf_crop = extract_region(surf_slice, region)
    atmos_crop = extract_region(atmos_slice, region)
    static_crop = extract_region(static_ds, region)

    return xr_to_batch(surf_crop, atmos_crop, static_crop)

def get_ground_truth_batches(start_time, region, dataset_loader, steps=56):
    targets = []
    for i in range(steps):
        t = start_time + timedelta(hours=6 * i)
        surface_ds_t, atmospheric_ds_t, static_ds_t = dataset_loader(t)

        surface_region = extract_region(surface_ds_t, region)
        atmos_region = extract_region(atmospheric_ds_t, region)
        static_region = extract_region(static_ds_t, region)

        batch = xr_to_batch(surface_region, atmos_region, static_region)
        targets.append(batch)
    return targets



def root_mean_squared_error(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))


# ------------------ DATASET LOADER FUNCTION ------------------ #
def dataset_loader(t: datetime):
    time_window = [t - timedelta(hours=6), t]
    surface_ds_t = surface_ds.sel(time=time_window, method="nearest", tolerance=np.timedelta64(1, "h"))
    atmospheric_ds_t = atmospheric_ds.sel(time=time_window, method="nearest", tolerance=np.timedelta64(1, "h"))
    return surface_ds_t, atmospheric_ds_t, static_ds


# ------------------ EVALUATION FUNCTION ------------------ #
@torch.no_grad()
def evaluate_14_day_rollout(model, start_batch, get_ground_truth_batches_fn, climatology_ds, patch_size=16, tolerance=2.0):
    model.eval()
    step = timedelta(hours=6)
    device = next(model.parameters()).device

    predictions = []
    current_input = deepcopy(start_batch).to(device)

    for _ in range(56):
        pred = model(current_input, step)
        pred = pred.crop(patch_size)
        predictions.append(move_batch_to_cpu(pred))
        current_input = roll_forward(current_input, pred)

    targets = get_ground_truth_batches_fn()

    def init_dict(vars_): return {k: [] for k in vars_}
    surface_vars = predictions[0].surf_vars.keys()
    atmos_vars = predictions[0].atmos_vars.keys()

    rmse_surf, rmse_atmos = init_dict(surface_vars), init_dict(atmos_vars)
    acc_surf, acc_atmos = init_dict(surface_vars), init_dict(atmos_vars)
    acc_rate_surf, acc_rate_atmos = init_dict(surface_vars), init_dict(atmos_vars)

    for pred, target in zip(predictions, targets):
        doy = target.metadata.time[-1].timetuple().tm_yday

        for k in surface_vars:
            y_pred = pred.surf_vars[k].numpy().ravel()
            y_true = target.surf_vars[k][:, -1].numpy().ravel()
            rmse_surf[k].append(root_mean_squared_error(y_true, y_pred))

            climatology = climatology_ds[k].sel(dayofyear=doy).values.ravel()

            anomaly_pred = y_pred - climatology
            anomaly_true = y_true - climatology
            acc_val = np.corrcoef(anomaly_pred, anomaly_true)[0, 1] if anomaly_true.std() > 0 else 0.0
            acc_surf[k].append(acc_val)
            acc_rate_surf[k].append(np.mean(np.abs(y_pred - y_true) < tolerance))

        for k in atmos_vars:
            y_pred = pred.atmos_vars[k].numpy().ravel()
            y_true = target.atmos_vars[k][:, -1].numpy().ravel()
            rmse_atmos[k].append(root_mean_squared_error(y_true, y_pred))

            climatology = climatology_ds[k].sel(dayofyear=doy).values.ravel()
            anomaly_pred = y_pred - climatology
            anomaly_true = y_true - climatology
            acc_val = np.corrcoef(anomaly_pred, anomaly_true)[0, 1] if anomaly_true.std() > 0 else 0.0
            acc_atmos[k].append(acc_val)
            acc_rate_atmos[k].append(np.mean(np.abs(y_pred - y_true) < tolerance))

    def avg_dict(d): return {k: float(np.mean(v)) for k, v in d.items()}

    return {
        "rmse": {"surf": avg_dict(rmse_surf), "atmos": avg_dict(rmse_atmos)},
        "acc": {"surf": avg_dict(acc_surf), "atmos": avg_dict(acc_atmos)},
        "accuracy": {"surf": avg_dict(acc_rate_surf), "atmos": avg_dict(acc_rate_atmos)}
    }


In [15]:
print("Initializing model...")
base = Xaurora(use_lora=False, autocast=True)
base.load_checkpoint("microsoft/aurora", "aurora-0.25-pretrained.ckpt", strict=False)
base.eval().cuda()
print("Model loaded and moved to GPU.")

region = "western_med"
start_time = datetime(2022, 1, 1, 0)
print(f"Region: {region}, Start Time: {start_time}")

print("Generating start batch...")
start_batch = get_start_batch(surface_ds, atmospheric_ds, static_ds, region, start_time)
print("Start batch ready.")

print("Preparing ground truth batches loader...")
get_gt_batches = lambda: get_ground_truth_batches(start_time, region, dataset_loader)

print("Running 14-day evaluation rollout...")
results = evaluate_14_day_rollout(base, start_batch, get_gt_batches, climatology_ds=climatology_ds, patch_size=8)

print("Evaluation complete. Displaying results...")

for metric_name in ["rmse", "acc", "accuracy"]:
    print(f"\n{metric_name.upper()}")
    for category in ["surf", "atmos"]:
        print(f"  {category.capitalize()}")
        for var, value in results[metric_name][category].items():
            print(f"    {var:>4}: {value:.4f}")


Initializing model...
Model loaded and moved to GPU.
Region: western_med, Start Time: 2022-01-01 00:00:00
Generating start batch...
Start batch ready.
Preparing ground truth batches loader...
Running 14-day evaluation rollout...
Evaluation complete. Displaying results...

RMSE
  Surf
     10u: 4.0450
     10v: 3.5327
      2t: 5.4448
     msl: 752.9736
  Atmos
       t: 4.2945
       u: 10.2784
       v: 10.2324
       q: 0.0014
       z: 831.9116

ACC
  Surf
     10u: 0.7647
     10v: 0.6470
      2t: 0.4697
     msl: 0.5525
  Atmos
       t: 0.7662
       u: 0.8139
       v: 0.6651
       q: 0.5514
       z: 0.7574

ACCURACY
  Surf
     10u: 0.4767
     10v: 0.5015
      2t: 0.3348
     msl: 0.0012
  Atmos
       t: 0.4105
       u: 0.2047
       v: 0.2119
       q: 1.0000
       z: 0.0017
